### Chatbot logic and implementation

In [1]:
import pandas as pd
import sqlite3
from datetime import datetime, timedelta
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
class BookChat:
    def __init__(self):
        self.books = pd.read_csv("./data/books.csv")
        self.orders = pd.read_csv("./data/orders.csv")
        self.customers = pd.read_csv('./data/customers.csv')

    def reply(self, msg: str):
        print(f"\nBookChat: {msg}")

    def user_msg(self, msg: str):
        print(f"\nUser: {msg}")

    def input(self, msg: str):
        self.reply(msg)
        user_input = input("\nUser: ")
        self.user_msg(user_input) # Only keep in VSCode
        return user_input


In [3]:
class Books(BookChat):
    def __init__(self):
        super().__init__()

    def get_book_details(self, book_title: str):
        return self.books[self.books["title"] == book_title].iloc[0]

    def check_availability(self):
        user_input = self.input(msg="What book are you looking for?")
        self.reply(msg="One moment, let me check the inventory.")

        if user_input:
            book = self.books[self.books["title"].str.contains(user_input, case=False)]

            if book.empty:
                self.reply(
                    f"Sorry, it seems that we do not have any book with the title '{user_input}' in stock."
                )

                return []

            else:
                self.reply(
                    f"Yes, '{book.iloc[0]['title']}' by {book.iloc[0]['author']} is currently in stock."
                )

                return book.iloc[0]
        else:
            self.reply("Sorry, I did not catch that. Could you please start again?")
            self.check_stock()

    def check_inventory(self, book):
        self.user_msg("How many copies are available?")

        self.reply("One moment, let me check the inventory.")

        self.reply(f"We currently have {book['quantityInStock']} copies in stock.")

    def check_price(self, book):
        self.user_msg("What is the price of the book?")

        self.reply("Let me find that information for you...")

        self.reply(
            f"{book['title']} by {book['author']} is priced at ${book['price']}."
        )

    def generate_cosine_matrix(self):
        # Create a Tfibooks.booksVectorizer object and combine all columns
        tfidf = TfidfVectorizer(stop_words="english")

        columns_to_combine = [
            "title",
            "author",
            "genre",
            "short_description",
            "publisher",
            "price",
        ]
        self.books["combined"] = (
            self.books[columns_to_combine]
            .apply(lambda x: " ".join(x.astype(str)), axis=1)
            .str.lower()
        )

        tfidf_matrix = tfidf.fit_transform(self.books["combined"])

        # Compute the cosine similarity matrix
        cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

        return cosine_sim

    # Function that takes in book feature as input and outputs most similar books
    def get_recommendations(self, feature_name, feature_query, cosine_sim):
        try:
            # Get the index of the book that matches the title
            idx = self.books.loc[
                self.books[feature_name].str.lower() == feature_query.lower()
            ].index[0]

            # Get the pairwsie similarity scores of all books with that book
            sim_scores = list(enumerate(cosine_sim[idx]))

            # Sort the books based on the similarity scores
            sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

            # Get the scores of the 5 most similar books
            sim_scores = sim_scores[1:6]

            # Get the book indices
            book_indices = [i[0] for i in sim_scores]

            # Return the top 5 most similar books
            return self.books["title"].iloc[book_indices]
        except IndexError:
            # print(f"Error: {title} not found in our inventory.")
            return False

    def ask_for_recommendation(self):
        self.user_msg("Can you recommend a book for me?")
        rec_choice = self.input(
            msg="Sure, do you want a recommendation by title or genre? (Title/Genre)"
        ).lower()

        if "title" in rec_choice:
            return "title", self.input(
                msg="What is the title of the book you are looking for?"
            )
        elif "genre" in rec_choice:
            return "genre", self.input(msg="What genre are you interested in?")
        else:
            self.reply("Sorry, I did not catch that. Could you please start again?")
            return self.ask_for_recommendation()

    def show_recommendations(self, feature_name, feature_query):
        cosine_sim = self.generate_cosine_matrix()

        recommended_book = self.get_recommendations(
            feature_name, feature_query, cosine_sim
        )

        if recommended_book is not False:
            rec_list = "\n".join(
                [f"{i+1}. {book}" for i, book in enumerate(recommended_book)]
            )

            self.reply(f"Here are some books you might like: \n{rec_list}.")

            return recommended_book
        else:
            return self.reply(
                "Sorry, I could not find any books that match your preference."
            )

    def query_recommendations(self, recommendation_list):
        self.user_msg("Sounds interesting! Do you have the 2nd book in stock?")
        self.reply("Let me check the inventory...")

        book_details = self.get_book_details(recommendation_list.iloc[1])

        self.reply(
            f"We currently have {book_details['quantityInStock']} copies of {book_details['title']} in stock."
        )

        self.user_msg("What is the price of that book?")
        self.reply("Let me find that information for you...")

        self.reply(f"{book_details['title']} is priced at ${book_details['price']}.")

        return

In [4]:
class Orders(BookChat):
    def __init__(self):
        super().__init__()

    def get_orders(self):
        return self.orders

    def reserve_copy(self, book, customer):
        self.user_msg("Can you reserve a copy for me?")

        # Get the maximum orderID value
        max_order_id = self.orders["orderID"].max()

        # Increment the maximum orderID value to get the next available ID
        next_order_id = max_order_id + 1

        # Update orders table with new orderID
        new_order = {
            "orderID": next_order_id,
            "bookID": book["bookID"],
            "customerID": customer["customerID"],
            "orderDate": datetime.now().strftime("%Y-%m-%d"),
            "pickupDate": (datetime.now() + timedelta(days=2)).strftime("%Y-%m-%d"),
            "status": "Reserved",
        }

        self.orders = pd.concat(
            [self.orders, pd.DataFrame([new_order])], ignore_index=True
        )
        self.orders.to_csv("./data/orders_DUMP.csv", index=False)

        # Reduce quantityInStock by 1
        self.books.loc[self.books["bookID"] == book["bookID"], "quantityInStock"] -= 1
        self.books.to_csv("./data/books_DUMP.csv", index=False)

        self.reply(
            f"Thank you {customer['firstName']}, I have reserved a copy of {book['title']} for you. Please visit the store within the next 48 hours to pick it up."
        )

In [5]:
class Customers(BookChat):
    def __init__(self):
        super().__init__()

    def get_customer(self, customerID):
        try:
            return self.customers[self.customers["customerID"] == customerID].iloc[0]
        except IndexError:
            return None

    def add_customer(self):
        self.reply(
            "Welcome to Argo! We hope you have a great experience with us. To help us assist you better, please provide a few details about yourself."
        )
        # Get max customer ID
        max_id = self.customers["customerID"].max()

        # Ask for name, address, phone number, email
        firstName = self.input("What is your first name?")
        lastName = self.input("What is your last name?")
        phone = self.input("What is your phone number?")
        email = self.input("What is your email address? (Leave blank if none)")

        new_customer = {
            "customerID": max_id + 1,
            "firstName": firstName,
            "lastName": lastName,
            "phone": phone,
            "email": email,
        }

        self.customers = pd.concat(
            [self.customers, pd.DataFrame([new_customer])], ignore_index=True
        )

        self.customers.to_csv("./data/customers_DUMP.csv", index=False)

        self.reply(f"Thank you for shopping with us! Your customer ID is {max_id + 1}.")
        self.reply("Please use this customer ID for future purchases and queries.")
        self.reply(f"What can I help you with, {firstName}?")

        return self.get_customer(max_id + 1)

In [6]:
class Store(BookChat):
    def __init__(self):
        super().__init__

    def store_hours(self):
        self.user_msg("What are your store hours?")
        self.reply(
            "Our store is open from 11 AM to 7 PM from Monday to Friday. On Saturdays and Sundays, we are open from 12 PM to 5 PM."
        )

    def store_location(self):
        self.user_msg("Where is your store located?")
        self.reply(
            "We are located at 1841-A Saint-Catherine St W, Montreal, QC H3H 1M2."
        )

    def areas_served(self):
        self.user_msg("What areas do you serve?")
        self.reply(
            "We have only one store in Montreal, but we ship anywhere in the Island of Montreal, as well as Laval and South Store."
        )

    def delivery(self):
        self.user_msg("Do you deliver books?")
        self.reply(
            "Yes, we do deliver books in the Island of Montreal. Local delivery is a flat rate of $8.50 (plus tax), but it's free for any purchase of $100 or more."
        )

In [8]:
chatbot = BookChat()
books = Books()
orders = Orders()
customers = Customers()
store = Store()

### Welcome

In [9]:
chatbot.reply("Hello, I'm BookChat by Argo Bookstore!")

customer_type = chatbot.input(
    "Are you a new customer or an existing customer? (New/Existing)"
)

if "existing" in customer_type:
    while True:
        customerInfo = customers.get_customer(
            int(
                chatbot.input(
                    "Welcome back! Please provide your customer ID so that we can provide you with a personalized experience!"
                )
            )
        )

        if customerInfo is not None:
            chatbot.reply(f"Welcome back, {customerInfo['firstName']}! What can I help you with?")
            break

        else:
            chatbot.reply("Sorry, I couldn't find your customer ID. Please try again.")

elif "new" in customer_type:
    customerInfo = customers.add_customer()

else:
    chatbot.reply("Sorry, I didn't understand that.")


BookChat: Hello, I'm BookChat by Argo Bookstore!

BookChat: Are you a new customer or an existing customer? (New/Existing)

User: 

BookChat: Sorry, I didn't understand that.


### Scenario 1: Book availablity and booking

In [ ]:
bookSearched = books.check_availability()

In [10]:
books.check_inventory(bookSearched)
books.check_price(bookSearched)


User: How many copies are available?

BookChat: One moment, let me check the inventory.

BookChat: We currently have 5 copies in stock.

User: What is the price of the book?

BookChat: Let me find that information for you...

BookChat: Moby-Dick by Herman Melville is priced at $11.99.


In [11]:
orders.reserve_copy(bookSearched, customerInfo)


User: Can you reserve a copy for me?

BookChat: Thank you Emily, I have reserved a copy of Moby-Dick for you. Please visit the store within the next 48 hours to pick it up.


### Scenario 2: Book Recommendation

In [13]:
# Recommend by title or by genre, depending on user input
recommend_question = books.ask_for_recommendation()


User: Can you recommend a book for me?

BookChat: Sure, do you want a recommendation by title or genre? (Title/Genre)

User: title

BookChat: What is the title of the book you are looking for?

User: the iliad


In [14]:
recommendation_list = books.show_recommendations(*recommend_question)


BookChat: Here are some books you might like: 
1. The Odyssey
2. The Divine Comedy
3. Pride and Prejudice
4. The Count of Monte Cristo
5. A Farewell to Arms.


In [14]:
books.query_recommendations(recommendation_list)


User: Sounds interesting! Do you have the 2nd book in stock?

BookChat: Let me check the inventory...

BookChat: We currently have 130 copies of The Big Two-Hearted River in stock.

User: What is the price of that book?

BookChat: Let me find that information for you...

BookChat: The Big Two-Hearted River is priced at $11.99.


### Scenario 3: Store Information Inquiry

In [11]:
store.store_location()


User: Where is your store located?

BookChat: We are located at 1841-A Saint-Catherine St W, Montreal, QC H3H 1M2.


In [12]:
store.store_hours()


User: What are your store hours?

BookChat: Our store is open from 11 AM to 7 PM from Monday to Friday. On Saturdays and Sundays, we are open from 12 PM to 5 PM.


In [13]:
store.areas_served()


User: What areas do you serve?

BookChat: We have only one store in Montreal, but we ship anywhere in the Island of Montreal, as well as Laval and South Store.


In [14]:
store.delivery()


User: Do you deliver books?

BookChat: Yes, we do deliver books in the Island of Montreal. Local delivery is a flat rate of $8.50 (plus tax), but it's free for any purchase of $100 or more.


### Scenario 5: Handling Customer Feedback

In [40]:
import pandas as pd
from textblob import TextBlob

# Dummy order data
orders = pd.read_csv("./data/orders.csv")

# Create a DataFrame to store feedback
feedback_df = pd.DataFrame(columns=["OrderID", "Feedback", "Sentiment"])

In [42]:
def get_order_details(order_id):
    order_details = orders[orders["orderID"] == order_id].drop(columns=["status"], axis=1)
    return order_details.to_dict(orient="records")[0]

def perform_sentiment_analysis():
    global feedback_df
    sentiment_values = []

    for feedback in feedback_df["Feedback"]:
        analysis = TextBlob(feedback)
        sentiment = "Positive" if analysis.sentiment.polarity > 0 else "Negative" if analysis.sentiment.polarity < 0 else "Neutral"
        sentiment_values.append(sentiment)
    
    feedback_df["Sentiment"] = sentiment_values

def process_feedback(order_id, feedback):
    global feedback_df
    new_feedback = pd.DataFrame({"OrderID": [order_id], "Feedback": [feedback]})
    feedback_df = pd.concat([feedback_df, new_feedback], ignore_index=True)
    perform_sentiment_analysis()

def main():
    print("Welcome to the Bookstore Chatbot!")

    while True:
        order_id = int(input("Please provide your order ID: "))
        order_details = get_order_details(order_id)

        if order_details is None:
            print("I'm sorry, I couldn't find your order. Could you please confirm the order ID again?")
            continue
        else:        
            print("Order Details:")
            for key, value in order_details.items():
                print(f"- {key.capitalize()}: {value}")

        confirmation = input("Are these order details correct? (yes/no): ")
        if confirmation.lower() != "yes":
            print("I'm sorry for the inconvenience. Could you please provide the order ID again.")
            continue

        print("How can I assist you today?",flush=True)
        print("1. Request a Replacement",flush=True)
        print("2. Initiate a Return",flush=True)
        print("3. Provide Feedback",flush=True)

        while True:
            choice = input("Please select an option (1/2/3): ")

            if choice == "1":
                print("You have chosen to request a replacement.",flush=True)
                reason = input("Please provide the reason for the replacement: ")
                shipping_choice = input("Would you like us to ship the replacement? (yes/no): ")
                if shipping_choice.lower() == "yes":
                    print("Thank you for your input. Your return request has been approved, we'll arrange for a replacement to be shipped and email the details shortly.")
                else:
                    print("Thank you for your input. Your replacement request has been approved, you will receive a confirmation email soon. Please visit the store to collect the replacement.")
                print("A confirmation receipt will be sent to your email after the process.")
                break
            elif choice == "2":
                print("You have chosen to initiate a return.",flush=True)
                reason = input("Please provide the reason for the return: ")
                shipping_choice = input("Would you like us to arrange for pickup or visit us? (pickup/visit): ")
                if shipping_choice.lower() == "pickup":
                    print("Thank you for your input. Your return request has been approved, we'll arrange for a pickup of the item and email the details shortly.")
                else:
                    print("Thank you for your input. Your return request has been approved, you will receive a confirmation email soon. Please visit the store to initiate the return.")
                break
            elif choice == "3":
                print("You have chosen to provide feedback.",flush=True)
                feedback = input("Please share your feedback: ")
                process_feedback(order_id, feedback)
                print("Feedback recorded. Thank you for sharing your thoughts!")
                break
            else:
                print("Invalid choice. Please select a valid option.")

        another_action = input("Action completed. Would you like to perform another action? (yes/no): ")
        if another_action.lower() != "yes":
            print("Thank you for using our chatbot. Have a great day!")
            break

#if __name__ == "__main__":
#    main()